# Anomaly Detectin: Evaluation

Here, we will import the test data or the predicted layer masks and evaluate our model using them.

Import necessary libraries and mount the drive for data access.

In [1]:
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the Model

Load the saved model.

In [3]:
project_loc = '/content/drive/MyDrive/OCTA500/Project/Anomaly/'
model = load_model(project_loc + 'best_model.h5')

## Evaluate

In [9]:
def evaluate_in_batches(model, X, y, batch_size=32, threshold=0.5):
    total_samples = len(X)
    correct_predictions = 0
    for i in range(0, total_samples, batch_size):
        batch_X = X[i:i + batch_size]
        batch_y = y[i:i + batch_size]
        predictions = model.predict(batch_X)
        predicted_labels = (predictions > threshold).astype(int)
        correct_predictions += np.sum(predicted_labels.flatten() == batch_y)

    accuracy = correct_predictions / total_samples
    return accuracy

### Test Data

Load the test data and evaluate the model.

In [7]:
data_loc = project_loc + 'data/'

# Load the testing data
test_data = np.load(data_loc + 'mt_test.npz')
X_test = test_data['mask']  # Masks
y_test = test_data['text']  # Texts

# Slice amount
slices = X_test.shape[1]

# Reshape the layers for CNN
X_test = X_test.reshape(-1, slices, 512, 512, 6)

In [10]:
batch_size = 4
accuracy = evaluate_in_batches(model, X_test, y_test, batch_size)
print(f'\033[91mAccuracy: \033[1m{accuracy}\033[0m')

1/1 [==============================] - 0s 245ms/step
Accuracy: 0.6933333333333334


## See Results Individually

Here, you can set a sample in the range given below, and test for its output.

In [16]:
range_index = X_test.shape[0]
print(f'Range: (0, {range_index})')

def eval(index, model, threshold=0.5):
    index = abs(index)
    if index > range_index:
        print(f'\033[91mIndex {index} out of range (0, {range_index})\033[0m')
        return -1, -1, -1

    sample = X_test[index]
    sample = np.expand_dims(sample, axis=0)

    sample_label = y_test[index]
    prediction = model.predict(sample)
    predicted_label = (prediction > threshold).astype(int)

    return sample_label, prediction[0][0], predicted_label[0][0]

Range: (0, 75)


In [19]:
# Set index here.
index = 5

sample_label, prediction, predicted_label = eval(index, model)
print(f'\033[1mSample {index}:\033[0m')
print(f'Original Label:  \033[91m{sample_label}\033[0m')
print(f'Prediction:      \033[91m{prediction:.4f}\033[0m')
print(f'Predicted Label: \033[91m{predicted_label}\033[0m')

1/1 [==============================] - 0s 409ms/step
Sample 5:
Original Label:  1
Prediction:      0.6354
Predicted Label: 1
